# Redis Cluster Tutorial

This notebook demonstrates how to work with Redis Cluster using Python. We'll explore key distribution, sharding, and failover mechanisms.

## Setup

First, let's install the required packages and establish a connection to our Redis Cluster.

In [ ]:
%%capture
%pip install redis rich tqdm

In [ ]:
import redis
from rich.pretty import pprint

In [ ]:
# Creating cluster connection
# Note: Make sure your Jupyter server is connected to the Redis network
# Run this command if needed: `docker network connect redis_default jupyter-jupyter-1`
r = redis.RedisCluster(host='master', port=6379)
print("Connected to Redis Cluster")

## Basic Redis Operations

Let's start with some basic operations to store and retrieve data from our Redis Cluster.

In [ ]:
# Set two String keys
result1 = r.set("foo", "bar")
result2 = r.set("hello", "world")

print(f"Key 'foo' set successfully: {result1}")
print(f"Key 'hello' set successfully: {result2}")

In [ ]:
# Retrieve keys
foo_value = r.get("foo")
hello_value = r.get("hello")

print(f"Key 'foo' value: {foo_value}")
print(f"Key 'hello' value: {hello_value}")

## Understanding Redis Cluster Sharding

Redis Cluster distributes keys across different nodes using a hash slot mechanism. Each key is assigned to one of 16384 hash slots, and these slots are distributed among the cluster nodes.

Let's examine which slots our keys are mapped to:

In [ ]:
# Retrieve keys' slots
foo_slot = r.keyslot("foo")
hello_slot = r.keyslot("hello")

print(f"Key 'foo' is mapped to slot: {foo_slot}")
print(f"Key 'hello' is mapped to slot: {hello_slot}")

Now, let's examine the cluster's sharding configuration to understand how the nodes are distributed:

In [ ]:
# Get detailed cluster sharding information
shards = r.cluster_shards()
print("Complete cluster sharding configuration:")
pprint(shards)

The output above shows the raw cluster sharding information, which can be quite verbose. Let's create a cleaner representation of the cluster nodes:

In [ ]:
# Create a cleaner view of sharding information
import socket

def get_clean_nodes_list():
    """Return a clean list of nodes in the Redis Cluster"""
    shards = r.cluster_shards()
    nodes = []
    
    for shard in shards:
        for node in shard["nodes"]:
            try:
                # Try to resolve IP to hostname
                hostname = socket.gethostbyaddr(node[b"ip"])[0]
            except:
                hostname = node[b"ip"]
                
            nodes.append({
                "node": hostname,
                "role": node[b"role"],
                "health": node[b"health"],
                "replication-offset": node[b'replication-offset'],
                "shards": shard['slots']
            })
    return nodes

nodes = get_clean_nodes_list()
print("Cluster nodes and their assigned slots:")
pprint(nodes)

## Testing Redis Cluster Failover

One of the main advantages of Redis Cluster is high availability through replication and automatic failover. 

Let's first check which node is responsible for our 'foo' key:

In [ ]:
# Check which slot the 'foo' key is mapped to
foo_slot = r.keyslot("foo")
print(f"Key 'foo' is mapped to slot: {foo_slot}")

# Find which node handles this slot
for node in nodes:
    for slot_range in node["shards"]:
        if slot_range[0] <= foo_slot <= slot_range[1]:
            print(f"Key 'foo' is handled by node: {node['node']} with role: {node['role']}")

### Simulating a Node Failure

In a real environment, you can test failover by stopping a master node. When you do this, a replica should be automatically promoted to master.

For example, if we determine that our 'foo' key is handled by redis-master-3, we could run:
```bash
docker stop redis-master-3
```

Let's see if we can still access our data after a simulated node failure:

In [ ]:
# Try to get the value after the node handling this key might be down
# If failover worked correctly, we should still get our value
try:
    value = r.get("foo")
    print(f"Successfully retrieved 'foo' value: {value}")
except Exception as e:
    print(f"Error retrieving key: {e}")

## Examining Cluster State After Failover

After a failover event, let's check the new cluster state to see the changes in node roles:

In [ ]:
# Get updated cluster node information
updated_nodes = get_clean_nodes_list()
print("Updated cluster configuration after failover:")
pprint(updated_nodes)

## Conclusion

In this tutorial, we've explored:

1. Basic operations with Redis Cluster
2. How Redis Cluster maps keys to slots
3. How to examine the cluster's node and shard configuration
4. Simulating and observing failover behavior

Redis Cluster provides a robust solution for distributing data across multiple nodes while maintaining high availability through replication and automatic failover.

## Further Resources

- [Redis Cluster Specification](https://redis.io/topics/cluster-spec)
- [Redis Cluster Tutorial](https://redis.io/topics/cluster-tutorial)
- [Python redis-py Client](https://redis-py.readthedocs.io/en/stable/)